# AutoCron Demo 3: Async Tasks

This notebook demonstrates AutoCron's native async/await support.

## Features Covered:
- Async function scheduling
- Concurrent async tasks
- Mixing sync and async tasks
- Async with retries and timeouts
- Real-world async examples

## Setup

In [12]:
from autocron import AutoCron, schedule
from datetime import datetime
import asyncio
import aiohttp
import time

## 1. Basic Async Task Scheduling

Schedule async functions just like regular functions!

In [13]:
@schedule(every='10s')
async def async_hello():
    """Simple async task."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Async hello started")
    await asyncio.sleep(2)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Async hello completed")
    return "Hello from async"

@schedule(every='15s')
async def async_counter():
    """Async task with multiple awaits."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Counting asynchronously")
    for i in range(1, 4):
        await asyncio.sleep(1)
        print(f"{i}...")
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Count complete")
    return "Counted to 3"

print("Async tasks scheduled")
print("These tasks use asyncio and run concurrently")

2026-02-22 15:44:49 - INFO - Task 'async_hello' scheduled with: interval=10s
2026-02-22 15:44:49 - INFO - Task 'async_counter' scheduled with: interval=15s


Async tasks scheduled
These tasks use asyncio and run concurrently


## 2. Async HTTP Requests

Make async API calls efficiently.

In [14]:
@schedule(every='30s')
async def fetch_weather():
    """Fetch weather data asynchronously."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Fetching weather data")

    async with aiohttp.ClientSession() as session:
        url = "https://api.openweathermap.org/data/2.5/weather?q=London&appid=demo"
        try:
            async with session.get(url, timeout=10) as response:
                if response.status == 200:
                    data = await response.json()
                    print(f"Weather fetched: {data.get('weather', 'N/A')}")
                    return data
                print(f"Status: {response.status}")
        except Exception as error:
            print(f"Error: {error}")

    return None

print("Async HTTP task scheduled")
print("Uses aiohttp for non-blocking requests")

2026-02-22 15:44:49 - INFO - Task 'fetch_weather' scheduled with: interval=30s


Async HTTP task scheduled
Uses aiohttp for non-blocking requests


## 3. Multiple Concurrent Async Tasks

Run multiple async operations in parallel.

In [15]:
@schedule(every='20s')
async def fetch_multiple_apis():
    """Fetch from multiple APIs concurrently."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Fetching from 3 APIs")

    async def fetch_api(name, delay):
        print(f"{name}: Starting")
        await asyncio.sleep(delay)
        print(f"{name}: Complete")
        return {"api": name, "status": "success"}

    results = await asyncio.gather(
        fetch_api("API-1", 2),
        fetch_api("API-2", 3),
        fetch_api("API-3", 1),
    )

    print(f"[{datetime.now().strftime('%H:%M:%S')}] All APIs fetched")
    return results

print("Concurrent async task scheduled")
print("All 3 APIs are fetched in parallel")

2026-02-22 15:44:49 - INFO - Task 'fetch_multiple_apis' scheduled with: interval=20s


Concurrent async task scheduled
All 3 APIs are fetched in parallel


## 4. Mixing Sync and Async Tasks

AutoCron handles both sync and async tasks seamlessly.

In [16]:
@schedule(every='10s')
def sync_task():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Sync task running")
    time.sleep(2)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Sync task done")
    return "Sync result"

@schedule(every='10s')
async def async_task():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Async task running")
    await asyncio.sleep(2)
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Async task done")
    return "Async result"

print("Both sync and async tasks scheduled")
print("AutoCron handles both task types")

2026-02-22 15:44:49 - INFO - Task 'sync_task' scheduled with: interval=10s
2026-02-22 15:44:49 - INFO - Task 'async_task' scheduled with: interval=10s


Both sync and async tasks scheduled
AutoCron handles both task types


## 5. Async with Retries

Combine async tasks with retry logic.

In [23]:
import random

@schedule(every='30s', retries=3, retry_delay=5)
async def unreliable_async_api():
    """Async API that might fail."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Calling unreliable API")
    await asyncio.sleep(1)

    if random.random() < 0.6:
        print("API request failed")
        raise Exception("Network timeout")

    print("API request succeeded")
    return {"status": "success", "data": "payload"}

print("Async task with retries scheduled")
print("Will retry up to 3 times on failure")

2026-02-22 15:45:05 - INFO - Task 'unreliable_async_api' scheduled with: interval=30s


Async task with retries scheduled
Will retry up to 3 times on failure


## 6. Async with Timeout

Prevent async tasks from running too long.

In [24]:
@schedule(every='25s', timeout=5)
async def slow_async_operation():
    """Async operation that might timeout."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting slow operation")

    duration = random.randint(3, 8)
    print(f"Will take {duration} seconds")
    await asyncio.sleep(duration)

    print(f"[{datetime.now().strftime('%H:%M:%S')}] Operation complete")
    return "Done"

print("Async task with 5-second timeout scheduled")
print("It will be terminated if it takes longer than 5 seconds")

2026-02-22 15:45:07 - INFO - Task 'slow_async_operation' scheduled with: interval=25s


Async task with 5-second timeout scheduled
It will be terminated if it takes longer than 5 seconds


## 7. Real-World Example: Web Scraping

Scrape multiple websites concurrently.

In [25]:
@schedule(every='5m')
async def scrape_news_sites():
    """Scrape multiple sites concurrently."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting news scraper")

    async def scrape_site(name, url):
        print(f"Scraping {name}")
        async with aiohttp.ClientSession() as session:
            try:
                async with session.get(url, timeout=10) as response:
                    html = await response.text()
                    await asyncio.sleep(1)
                    articles_count = len(html) // 1000
                    print(f"{name}: Found {articles_count} articles")
                    return {"site": name, "articles": articles_count, "status": "success"}
            except Exception as error:
                print(f"{name}: Error - {error}")
                return {"site": name, "status": "error"}

    results = await asyncio.gather(
        scrape_site("TechCrunch", "https://techcrunch.com"),
        scrape_site("Hacker News", "https://news.ycombinator.com"),
        scrape_site("Reddit", "https://www.reddit.com/r/programming"),
        return_exceptions=True,
    )

    print(f"[{datetime.now().strftime('%H:%M:%S')}] Scraping complete")
    return results

print("News scraper scheduled")
print("Scrapes 3 sites concurrently every 5 minutes")

2026-02-22 15:45:08 - INFO - Task 'scrape_news_sites' scheduled with: interval=5m


News scraper scheduled
Scrapes 3 sites concurrently every 5 minutes


## 8. Real-World Example: Database Operations

Perform async database operations.

In [26]:
@schedule(every='10m')
async def sync_databases():
    """Sync data between multiple databases."""
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Starting database sync")

    async def fetch_from_db(db_name):
        print(f"Fetching from {db_name}")
        await asyncio.sleep(2)
        records = random.randint(100, 1000)
        print(f"{db_name}: {records} records fetched")
        return {"db": db_name, "records": records}

    async def write_to_db(db_name, data):
        print(f"Writing to {db_name}")
        await asyncio.sleep(1)
        print(f"{db_name}: Write complete")
        return {"db": db_name, "status": "written", "rows": len(data)}

    source_data = await asyncio.gather(
        fetch_from_db("PostgreSQL"),
        fetch_from_db("MongoDB"),
        fetch_from_db("Redis"),
    )

    print("All data fetched, aggregating")
    await asyncio.sleep(1)
    result = await write_to_db("DataWarehouse", source_data)

    print(f"[{datetime.now().strftime('%H:%M:%S')}] Database sync complete")
    return result

print("Database sync scheduled")
print("Syncs data from 3 databases concurrently every 10 minutes")

2026-02-22 15:45:10 - INFO - Task 'sync_databases' scheduled with: interval=10m


Database sync scheduled
Syncs data from 3 databases concurrently every 10 minutes


## 9. Using Scheduler Class with Async

Add async tasks using the AutoCron class directly.

In [27]:
scheduler = AutoCron()

async def async_health_check():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Health check")
    await asyncio.sleep(1)
    return "Healthy"

async def async_metrics_collector():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] Collecting metrics")
    await asyncio.sleep(2)
    return {"cpu": 45, "memory": 62, "disk": 78}

scheduler.add_task(
    name="health_check",
    func=async_health_check,
    every='30s',
)

scheduler.add_task(
    name="metrics",
    func=async_metrics_collector,
    every='1m',
    timeout=10,
)

print("Async tasks added to scheduler")
print(f"Total tasks: {len(scheduler.list_tasks())}")

2026-02-22 15:45:11 - INFO - Task 'health_check' scheduled with: interval=30s
2026-02-22 15:45:11 - INFO - Task 'metrics' scheduled with: interval=1m


Async tasks added to scheduler
Total tasks: 2


## 10. Performance Comparison: Sync vs Async

See the speed difference between sync and async approaches.

In [28]:
import time

def sync_fetch_all():
    print("Synchronous (Sequential):")
    start = time.time()
    for i in range(5):
        time.sleep(1)
        print(f"API {i + 1} complete")
    elapsed = time.time() - start
    print(f"Total time: {elapsed:.2f} seconds\n")

async def async_fetch_all():
    print("Asynchronous (Concurrent):")
    start = time.time()

    async def fetch(i):
        await asyncio.sleep(1)
        print(f"API {i + 1} complete")

    await asyncio.gather(*[fetch(i) for i in range(5)])
    elapsed = time.time() - start
    print(f"Total time: {elapsed:.2f} seconds\n")

print("Performance Comparison: 5 API Calls\n")
print("=" * 50)

sync_fetch_all()
await async_fetch_all()

print("=" * 50)
print("Async is significantly faster for I/O-bound operations")

Performance Comparison: 5 API Calls

Synchronous (Sequential):
API 1 complete
API 2 complete
API 3 complete
API 4 complete
API 5 complete
Total time: 5.00 seconds

Asynchronous (Concurrent):
API 1 complete
API 3 complete
API 5 complete
API 2 complete
API 4 complete
Total time: 1.01 seconds

Async is significantly faster for I/O-bound operations


## Summary

In this demo, you learned:

 **Async Scheduling** - Schedule async functions with `@schedule` or `AutoCron`

 **Concurrent Operations** - Use `asyncio.gather()` for parallel execution

 **Async HTTP** - Make non-blocking API calls with `aiohttp`

 **Mixed Tasks** - Combine sync and async tasks seamlessly

 **Error Handling** - Retries and timeouts work with async tasks

 **Performance** - 5x+ speedup for I/O-bound operations

### When to Use Async:
- **API calls** - Multiple HTTP requests
- **Database queries** - Multiple DB operations
- **File I/O** - Reading/writing multiple files
- **Web scraping** - Scraping multiple pages

### When to Use Sync:
- **CPU-intensive** - Heavy calculations
- **Simple scripts** - Short, sequential operations
- **Legacy code** - No async support

### Next Steps:
- Check out `04_persistence.ipynb` for saving/loading tasks
- See `05_safe_mode.ipynb` for secure task execution
- Explore `06_dashboard.ipynb` for visual monitoring